# 第 6 章 逻辑斯谛回归与最大熵模型
## 6.2 最大熵模型
### 例 6.1
**假设随机变量$X$有5个取值$\{A,B, C, D, E\}$，要估计取各个值的概率$P(A),P(B),P(C),P(D),P(E)$**.
**解：由题意得，估计概率值满足约束：$P(A)+P(B)+P(C)+P(D)+P(E)=1 $**

满足，这个约束的概率分布有无穷多个。如果**没有任何其他信息**，仍要对概率分布进行估计，一个办法就是**认为这个分布中各个值的概率是相等的**：
$$
\begin{aligned}
P(A)=P(B)=P(C)=P(D)=P(E)=\frac{1}{5}
\end{aligned}
$$
这种**概率相等**的认知表示了对**事实**的无知，因为没有更多的信息，这种判断是合理的。

### 算法 6.1 改进的迭代尺度算法 IIS

输入：特征函数$f_1,f_2, \cdots,f_n$；经验分布$\hat P(X,Y)$，最大熵模型$P_w(y|x)$

输出：最优参数值$w^*_i$，最优模型$P_w^*$

1. 对所有$i \in \{1, 2, \cdots, n\}$，取**初值$w_i=0$**

2. 对每一个$i \in \{1, 2, \cdots, n\}$
   - 令$\delta_i$是方程$\frac{\partial B(\delta|w)}{\partial \delta_i}=0$的解
   - 更新$w_i$的值：$w_i \leftarrow w_i +\delta_i$
3. 如果不是所有的$w_i$都收敛，则重复步骤（2）

其中
$$
\begin{aligned}
\frac{\partial B(\delta|w)}{\partial \delta_i}&=\sum_{x,y}\hat P(x,y)f_i(x,y)-\sum_x\hat P(x)\sum_y P_w(x,y)f_i(x,y)\exp(\delta_if^\#(x,y))
\end{aligned}
$$

### 算法 6.2 最大熵模型学习得BFGS算法
输入：特征函数$f_1,f_2, \cdots, f_n$，经验分布$\hat P(x, y)$，目标函数$f(w)$，梯度$g(w)=\nabla f(w),\ f(w)=-\Psi(W)$，**精度要求$\epsilon$**。

输出：最优参数值$w^*$；最优模型$P_{w^*}(y|x)$

（1）选定初始点$w^{(0)}$，取$B_0$为**正定对称矩阵**，置$k=0$；

（2）计算$g_k = g(w^{(k)})$。若$||g_k||<\epsilon$，则**停止计算**，得$w^*=w^{(k)}$，否则转（3）

（3）由$B_kp_k=-g_k$求出$p_k$；

（4）**一维搜索**：求$\lambda_k$使得
$$
\begin{aligned}
f(w^{(k)}+\lambda_kp_k)=\mathop\min\limits_{\lambda\ge 0}f(w^{(k)}+\lambda _kp)
\end{aligned}
$$
（5）置$w^{(k+1)}=w^{(k)}+\lambda_k p_k$；

（6）计算$g_{k+1}=g(w^{(k+1)})$，若$||g_{k+1}||<\epsilon$，则**停止计算**，得$w^* = w^{(k+1)}$。否则，按下式求出$B_{k+1}$：
$$
\begin{aligned}
B_{k+1}=B_k +\frac{y_ky_k^T}{y_k^T\delta_k}-\frac{B_k\delta_k\delta_k^TB_k}{\delta^T_kB_k\delta_k}
\end{aligned}
$$
其中，$y_k= g_{k+1}-g_k,\delta_k=w^{(k+1)}-w^{(k)}$

（7）置$k=k+1$，转（3）。

In [72]:
# [Reference](https://blog.csdn.net/weixin_44972997/article/details/117980281)

import numpy as np
# 目标优化函数
fun = lambda x : 0.5 * x[0] ** 2 + x[1] ** 2 - x[0] * x[1] - 2 * x[0]
# 对定义得函数求梯度值，数据放入mat矩阵中
def fungradient(x):
    x = np.array(x)
    partial_x_0 = x[0][0] - x[1][0] - 2
    partial_x_1 = -1.0 * x[0][0] + 2.0 * x[1][0]
    # 将数据解释为矩阵
    gradient = np.mat([partial_x_0, partial_x_1])
    return gradient

def BGFS(fun, fungradient, x0, epsilon):
    # 迭代次数
    iteration = 5000
    # 步长
    step = 0.45
    # 常数 在 [0, 0.5] 之间
    sigma = 0.3
    # 第k次迭代
    k = 0
    # 正定矩阵 Bk: 初始值为单位矩阵
    Bk = np.mat([[1.0, 0.0], [0.0, 1.0]])
    while k < iteration :
        # (2) 计算 gk = g(w^k)
        gk = fungradient(x0)
        # 如果 范数小于epsilon， 则停止计算，得 w^* = w^(k)
        if np.linalg.norm(gk) < epsilon:
            break
        # (3) 由 Bk × pk = - gk ————>  pk = (Bk.I) × (-gk.T)
        pk = -Bk.I.dot(gk.T)
        # (4) 一维搜索: 求 λk 使得 f(w^k + λk×pk) = min f(wk + λp_k)
        m = 0
        mk = 0
        while m < 30 :
            if fun(x0 + (step ** m ) * pk) < fun(x0) + sigma * (step ** m) * gk.dot(pk):
                mk = m
                break
            m = m + 1
        # (5) w^(k+1) = w^k + λ_k × p_k
        x = x0 + (step ** mk) * pk
        # (6) 计算 g_(k+1) = g(w^(k+1))
        if np.linalg.norm(fungradient(x)) < epsilon:
            break
        difg = fungradient(x) - gk
        delta = x - x0
        Bk = Bk - (Bk * delta * delta.T * Bk) / (delta.T * Bk * delta) + (difg.T * difg) / (difg * delta)
        k = k + 1
        x0 = x
        print("第 {0} 次迭代，参数值为：{1}, 函数值为：{2}".format(k, x0.T, fun(x0)))
    return x, k

In [73]:
# 选择初始点
x0 = np.array([[1], [1]])
# BFGS迭代
x, k = BGFS(fun, fungradient, x0, 1e-6)
print("---------------------------------------")
print("迭代次数为 {0} , 最优点为 {1} , 最小值为 {2}".format(k, x.T, fun(x)))

第 1 次迭代，参数值为：[[1.9  0.55]], 函数值为：[[-2.7375]]
第 2 次迭代，参数值为：[[3.   1.25]], 函数值为：[[-3.6875]]
第 3 次迭代，参数值为：[[4.00110947 1.99852071]], 函数值为：[[-3.99999556]]
第 4 次迭代，参数值为：[[4.00003401 1.99998268]], 函数值为：[[-4.]]
---------------------------------------
迭代次数为 4 , 最优点为 [[4.00000001 2.00000001]] , 最小值为 [[-4.]]


## 习题
### 6.1 确认逻辑斯谛分布属于指数分布族
[Reference](https://blog.csdn.net/qq_41562704/article/details/99757050)
#### 指数分布族
**指数分布族中的分布形式如下**：
$$
\begin{aligned}
p(x|\eta)=h(x)\exp\{\eta^TT(x)-A(\eta)\}
\end{aligned}
$$
其中：
- $\eta$表示自然参数(normal parameter)，可以是向量形式
- $T(x)$为充分统计量（sufficient statistic）
- $A(\eta)$为累积函数（accumulate function）,作用是确保概率和为1
- $h(x)$为 潜在度量（underlying measure）
#### (1)证明：二项逻辑斯谛回归
二项逻辑斯蒂回归模型如下：
$$
\begin{aligned}
P(Y=0|x) &= \frac{1}{1+\exp(wx)}\\
P(Y=1|x) &= \frac{\exp(w\cdot x)}{1+\exp(w\cdot x)}
\end{aligned}
$$
可以组合为
$$
\begin{aligned}
P(Y=y|x)&=P(Y=0|x)^{(1-y)}\cdot P(Y=1)^y\\
&=\frac{1}{1+\exp(wx)}^{(1-y)}\cdot \frac{\exp(w\cdot x)}{1+\exp(w\cdot x)}^y
\end{aligned}
$$
对二项逻辑斯蒂回归模型的右端进行改造，改为以$e$为底的形式, 且令$\pi(x) = \exp(w\cdot x)$
$$
\begin{aligned}
P(Y=y|x)&=\exp\left\{\log \left(\frac{1}{1+\pi(x)}^{(1-y)}\cdot \frac{\pi(x)}{1+\pi(x)}^y\right)\right\}\\
&=\exp\left\{y\log\pi(x)-\log(1+\pi(x))\right\}
\end{aligned}
$$
根据此式，可得
$$
\begin{aligned}
h(y) &= 1\\
T(y) &= y \\
\eta &= \log \pi(x)\\
A(\eta) &= \log(1+\pi(x))
\end{aligned}
$$
**所以，可证二项逻辑斯蒂回归是指数分布族**
#### (2) 证明： 多项逻辑斯谛回归模型
多项逻辑斯谛回归模型简写如下
$$
\begin{aligned}
P(Y=k|x)&=\frac{\exp(w_k\cdot x)}{1+\sum_{k=1}^{K-1}\exp(w_k \cdot x)}, k=1,2,\cdots, K-1\\
P(Y=K|x)&=\frac{1}{1+\sum_{k=1}^{K-1}\exp(w_k \cdot x)}
\end{aligned}
$$
令$\pi(x)=\sum_{k=1}^{K-1}\exp(w_k \cdot x) + 1$，则多项逻辑斯谛回归模型表示如下
$$
\begin{aligned}
P(Y|x)&=\left[\frac{\exp(w_k + x)}{\pi(x)}\right]^{f(y)}\cdot \left[\frac{1}{\pi(x)}\right]^{1-f(y)}\\
f(y)&=\left\{ \begin{array}{l}
0, y=K\\
1, y<K
\end{array} \right.
\end{aligned}
$$
对右端进行改造，以$e$为底
$$
\begin{aligned}
P(Y|x)&=\exp\left\{f(y)\log\exp(w_k\cdot x)+f(y)\log \pi(x)\right\}\\
&=\exp\left\{f(y)(\log\exp(w_k\cdot x)+\log \pi(x))\right\}
\end{aligned}
$$
根据此式，可得
$$
\begin{aligned}
h(y) &= 1\\
T(y) &= f(y) \\
\eta &= \log \pi(x)+\log\exp(w_k \cdot x)\\
A(\eta) &= 0
\end{aligned}
$$
**所以，可证多项逻辑斯蒂回归是指数分布族**
**综上所述，逻辑斯谛分布是指数分布族**

### 6.2 写出逻辑斯谛回归模型学习的梯度下降算法
输入：训练数据集$T=\{(x_1,y_1),(x_2,y_2),\cdots,(x_N, y_N)\},x_i \in R^n,\ \ y_i=\{0, 1\}$，学习率：$\lambda$
输出：$w$
（1）选取初值 $w_0$，$k=0$
（2）二项逻辑斯谛回归模型的对数似然函数为
$
L(w_k)=\sum_{i=1}^N \{y_i(w_k\cdot x)-\log(1+\exp(w_k \cdot x))\}
$
（3）对数似然函数求偏导:$\frac{\partial L}{\partial w_k}$
（4）更新偏导：$w_k \leftarrow w_k + \lambda \cdot \frac{\partial L}{\partial w_k}$
（5）转置（2），同时更新$k=k+1$，直到满足条件为止。

### 6.3 写出最大熵模型学习的DFP算法。（关于一般的DFP算法参见附录B）
#### DFP 算法
输入：目标函数$f(x)$，梯度$g(x)=\nabla f(x)$，精度要求为$\epsilon$
输出：$f(x)$的极小点$x^*$
（1）选定初始点$x^{(0)}$，取$G_0$为正定对阵矩阵，置$k=0$
（2）计算$g_k = g(x^{(k)})$。若$||g_k||<\epsilon$，则停止计算，得近似解$x^*=x^{(k)}$；否则转（3）
（3）置$p_k = -G_k \cdot g_k$
（4）**一维搜索**：求$\lambda_k$使得：
$$
\begin{aligned}
f(x^{(k)}+\lambda_k\cdot p_k)=\mathop\min\limits_{\lambda \ge 0} f(x^{(k)}+\lambda\cdot p_k)
\end{aligned}
$$
（5）置$x^{(k+1)}=x^{(k)}+\lambda_k \cdot p_k$
（6）计算$g_{k+1}=g(x^{(k+1)})$。如果$||g_{k+1}||<\epsilon$，则停止计算，得到近似解$x^*=x^{(k+1)}$；否则，按照以下的方式计算**正定对阵矩阵$G_{k+1}$**
$$
\begin{aligned}
G_{k+1}=G_k + \frac{\delta_k \delta_k^T}{\delta_k^Ty_k} - \frac{G_ky_ky_k^TG_k}{y_k^TG_ky_k}
\end{aligned}
$$
（7）置$k=k+1$，转（3）。

In [76]:
import numpy as np
# 目标优化函数
fun = lambda x : 0.5 * x[0] ** 2 + x[1] ** 2 - x[0] * x[1] - 2 * x[0]
# 对定义得函数求梯度值，数据放入mat矩阵中
def g(x):
    x = np.array(x)
    partial_x_0 = x[0][0] - x[1][0] - 2
    partial_x_1 = -1.0 * x[0][0] + 2.0 * x[1][0]
    # 将数据解释为矩阵
    gradient = np.mat([[partial_x_0] ,[partial_x_1]])
    return gradient

def DFP(fun, g, x0, epsilon):
    # 迭代次数
    iteration = 5000
    # 步长
    step = 0.45
    # 常数 在 [0, 0.5] 之间
    sigma = 0.3
    # 第k次迭代
    k = 0
    # 正定矩阵 Bk: 初始值为单位矩阵
    Gk = np.mat([[1.0, 0.0], [0.0, 1.0]])
    while k < iteration :
        # (2) 计算 gk = g(w^k)
        gk = g(x0)
        # 如果 范数小于epsilon， 则停止计算，得 w^* = w^(k)
        if np.linalg.norm(gk) < epsilon:
            break
        # (3) 由 Gk × pk = - gk ————>  pk = (Bk.I) × (-gk.T)
        pk = -Gk.dot(gk)
        # (4) 一维搜索: 求 λk 使得 f(w^k + λk×pk) = min f(wk + λp_k)
        m = 0
        mk = 0
        minf = 1e10
        while m < 100 :
            if fun(x0 + (step ** m ) * pk) < minf:
                minf = fun(x0 + (step ** m ) * pk)
                mk = m
            m = m + 1
        # (5) w^(k+1) = w^k + λ_k × p_k
        x = x0 + (step ** mk) * pk
        # (6) 计算 g_(k+1) = g(w^(k+1))
        if np.linalg.norm(g(x)) < epsilon:
            break
        y = g(x) - gk
        delta = x - x0
        Gk = Gk + (delta * delta.T) / (delta.T * y) + (Gk * y * y.T * Gk) / (y.T * Gk * y)
        k = k + 1
        x0 = x
        print("第 {0} 次迭代，参数值为：{1}, 函数值为：{2}".format(k, x0.T, fun(x0)))
    return x, k

In [77]:
# 选择初始点
x0 = np.array([[1], [1]])
# BFGS迭代
x, k = DFP(fun, g, x0, 1e-6)
print("---------------------------------------")
print("迭代次数为 {0} , 最优点为 {1} , 最小值为 {2}".format(k, x.T, fun(x)))

第 1 次迭代，参数值为：[[1.9  0.55]], 函数值为：[[-2.7375]]
第 2 次迭代，参数值为：[[2.5 1.5]], 函数值为：[[-3.375]]
第 3 次迭代，参数值为：[[2.79760386 1.20335729]], 函数值为：[[-3.60036227]]
第 4 次迭代，参数值为：[[2.86630793 1.56267719]], 函数值为：[[-3.66190931]]
第 5 次迭代，参数值为：[[2.98556361 1.38724614]], 函数值为：[[-3.73159193]]
第 6 次迭代，参数值为：[[3.48039968 1.51183355]], 函数值为：[[-3.88035272]]
第 7 次迭代，参数值为：[[3.42561888 1.61513764]], 函数值为：[[-3.9079818]]
第 8 次迭代，参数值为：[[3.40601933 1.65721013]], 函数值为：[[-3.90969914]]
第 9 次迭代，参数值为：[[3.42013852 1.63618599]], 函数值为：[[-3.91048143]]
第 10 次迭代，参数值为：[[3.41571157 1.64549445]], 函数值为：[[-3.91076282]]
第 11 次迭代，参数值为：[[3.42415749 1.63434218]], 函数值为：[[-3.91105838]]
第 12 次迭代，参数值为：[[3.4182206  1.64500288]], 函数值为：[[-3.91127342]]
第 13 次迭代，参数值为：[[3.42752264 1.63235663]], 函数值为：[[-3.91144069]]
第 14 次迭代，参数值为：[[3.42115621 1.64350686]], 函数值为：[[-3.91173642]]
第 15 次迭代，参数值为：[[3.41936103 1.64669793]], 函数值为：[[-3.91174779]]
第 16 次迭代，参数值为：[[3.42054653 1.64466325]], 函数值为：[[-3.91175375]]
第 17 次迭代，参数值为：[[3.41997844 1.64566562]], 函数值为：[[-3.91

C:\Users\LiangHao\AppData\Local\Temp\ipykernel_20932\29933237.py:48: RuntimeWarning: invalid value encountered in true_divide
  Gk = Gk + (delta * delta.T) / (delta.T * y) + (Gk * y * y.T * Gk) / (y.T * Gk * y)


第 62 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 63 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 64 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 65 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 66 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 67 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 68 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 69 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 70 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 71 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 72 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 73 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 74 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 75 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 76 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 77 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 78 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 79 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 80 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 81 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 82 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 83 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 84 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 85 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]
第 86 次迭代，参数值为：[[nan nan]], 函数值为：[[nan]]


KeyboardInterrupt: 